## Bytes and Registers


_burton rosenberg, 29 june 2023_


### Table of contents.

1. <a href="#intro">Introduction</a>
1. <a href=#basenotation>Binary and other notations</a>
1. Exercise: Converting to binary
1. <a href=#bits>Ways to represent a register</a>
   1. <a href=#punchedtape>Punched tape</a>
   1. <a href=#srlatch>SR Latch</a>
   1. <a href=#codon>Codon sequences</a>
   1. <a href=#qubit>A qubit</a>
1. <a href=#symbol>Ways to encode a symbol</a>
   1. <a href=#paritycheck>Parity check</a>
   1. <a href=#repetition>Repetition code</a>
   1. <a href=#hamming>Hamming code</a>
   1. <a href=#correction>Extended Discussion</a>



### <a name=intro>Introduction</a>

Computers have mechanisms for memory, computation, and communication. In this section we look at memory. The basic building blocks of memory in the modern computer are the _register_ and the _byte_.


The byte is the basic building block the Random Access Memory (RAM), which is the focus of most dicussions about memory. It is fast, large, and easily accessed. A _bit_ is a storage element that can be one of two states, we will call them 0 or 1. A _byte_ is 8 bits together with each bit having its place in the byte, from the 0-th bit to the 7-th bit. 

That the byte has 8 bits is so common that I will just define a byte to be 8 bits, but in fact, early computers had diferent lenghts of bytes. For various reasons, 8 turned out the just the right size for the purpose.

Bytes are collected into a large, enummerated array called the RAM memory. Each byte in the array is known by its number in this array, called the byte's _address_. That each byte is equally accessible regardless of its address gives rise to the name _random_ access memory. 

__N.B.:__ This is mostly true. So much depends on the speed of RAM that there are techniques that make memory accesss faster by not uniformly accross all bytes. Examples are caches and NUMA machines.

There are several useful consequences of having integer addresses,

- Some data items need to be built from multiple bytes. By using addresses in sequence the address of the item and all its components are easiliy determined.
- Some data items are in repeating multiples, and integer arithmetic can be used for these items to offest to the particular item in the group.

We note that a byte has 256 possibel different bit combinations. This is often assocated with the integers 0 through 255. 


<div style="float:right;margin:2em;">
<img width="512" src="../images/TCPL-1ed-bytesize.png"></a>
</div>

A _register_ is a a number $n$ of bits taken together where each bit has its location in the collection, from bit 0 up to bit $n-1$. In this sense it generalizes the byte. However registers are found in the CPU and are addressed differently. Some register (the _Instrution Pointer_ (IP) and the _Stack Pointer_ (SP)) are known intrinsicly to some instructions executed by the CPU. Some are named, some are indexed by an integer. 

However a typical size for a register on a 64 bit machine is 64 bits. Older machine had small registers. The Intel 8085, in the early 1980's, had 16 bit registers. They were quicly enough replaced by such machines as the Intel 8086 with 32 bit registers. In this case, for backwards compatibility, each 32 register also can be identified by the upper and lower halves as 16 bit registers. 

### <a name=basenotation>Binary and other notations</a>

The use of binary was basically practical. The bit state would be based on some physical phenomena, and so there would be noise. A binary indicator can be made very tolerant of noise but having a threshold in the reading of the phenomena so that about the threashod is a 1, and below a 0. The noise amounts woud not matter as long as the noise amplitude kept the intended signal on the proper side of the threshold.

Furthermore, the correspondence between the bit pattern and an integer can be made completely natural thorugh the binary representation of the integer value. Recall that a number $n$  can be represented as a collection of zero-one values, $b_i$ by,

$$
n = \sum_i b_i \, 2^i
$$

In the case of the byte, the bits are assigned their bit number, giving the $b_i$ that is to appear in this equation. 


Note that a byte is an integer that has exactly 8 $b_i$. We will follow C language syntax and indicate a binary number by prefixing a 0-1 string with the indicatior `0b`. The number 5 in binary is `0b101`. However thinking about a byte it is best to visualize this written as `0b0000010`, accounting for all 8 bits in the byte.

Already we see how tedious it is to write numbers in binary, although they suit the hardware just fine. Instead, when trying to thing about an integer in terms of which bits are 1 and which are 0, the number is better written in base 16, known as _hexidecimal_. It is pretty important for a computer scientist to know hexidecimal, so I hope to give you some exericses to practice this number notation.

Hexidecimal is base 16, hence there must be 16 "digits" for each place, and these are, 

$$
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, a, b, c, d, e, f
$$

We will fillow C language syntax and indicate a hexidecimal number by prefixing the string of thise hex-its with the indicator `0x`. The number 5 in hexidecimal is `0x5`, However thinking about a byte it is best to write this as `0x05`, so all 8 bits are accounted for.


We will call _register_ the notion of a byte, but of any length. This is novel language, except that it is consistent with the name _registers_ are is it used in CPU architectures. When data is stored in a CPU it is stored in a CPU register. A 16 bit register (Intel 8085) it can store numbers from `0x0000` to `0xffff` (0 to 65,536).


In [4]:
%%file count-up.c

/*
 * Printing different bases.
 * Code for printing in binary
 */

#include<stdio.h>
#include<stdlib.h>

void print_binary_aux(int i) {
    int bit ;
    if (i==0) return ;
    bit = i%2 ;
    print_binary_aux((i-bit)/2) ;
    printf("%1d", bit) ;
    return ;
}

void print_binary(int i) {
    if (i==0) printf("0b%d", i) ;
    else {
        printf("0b") ;
        print_binary_aux(i) ;
    }
}

int main(int argc, char * argv[]){
    int i ;
    int N = atoi(argv[1]) ;
    for (i=0;i<N;i++) {
        printf("%d\t", i) ;
        print_binary(i) ;
        printf("\t0x%d", i) ;
        printf("\n") ;
    }
    return 0 ;
}


Overwriting count-up.c


In [5]:
%%bash
S=count-up
cc -o $S $S.c
./$S 27
rm $S $S.c

0	0b0	0x0
1	0b1	0x1
2	0b10	0x2
3	0b11	0x3
4	0b100	0x4
5	0b101	0x5
6	0b110	0x6
7	0b111	0x7
8	0b1000	0x8
9	0b1001	0x9
10	0b1010	0x10
11	0b1011	0x11
12	0b1100	0x12
13	0b1101	0x13
14	0b1110	0x14
15	0b1111	0x15
16	0b10000	0x16
17	0b10001	0x17
18	0b10010	0x18
19	0b10011	0x19
20	0b10100	0x20
21	0b10101	0x21
22	0b10110	0x22
23	0b10111	0x23
24	0b11000	0x24
25	0b11001	0x25
26	0b11010	0x26



### <a name=bits>Ways to represent a register</a>

<div style="float:right;margin:2em;">
<a title="Billie Grace Ward from New York, USA, CC BY 2.0 &lt;https://creativecommons.org/licenses/by/2.0&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Paper_Tape_Drive_(31437412070).jpg"><img width="334" alt="Paper Tape Drive (31437412070)" src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/6f/Paper_Tape_Drive_%2831437412070%29.jpg/512px-Paper_Tape_Drive_%2831437412070%29.jpg"></a>
</div>


#### <a name=punchedtape>Punched tape</a>

A bit can be represented by any manner in which there can be a distinction between two states. Here we see an early data storage strategy with a paper tape. Each column across the tape was a 5 bit byte, and it locations marked off in 5-th way from edge to edge, a hole was a 1 and no hoe was a 0. The 5 bit byte was supported by an early code called the [Baudot code](https://en.wikipedia.org/wiki/Baudot_code), invented by Emile Baudot in 1870. Its 32 different combinations barely fit enough symbols to be useful. While the Baudo code is obsolete, the word _baud_ is still with us, and it refers to the number of symbols per second in a communication channel.

The code for this punched tape might rather be the Murray code, 1901, which modified the Baudot code to minimize the average numbers of holes punched given a typical message. 

Both these codes use a system of _shift_, where a shift character uses the reminaing 31 characters in either _letter_ or _figure_ contexts. This idea is still used today, for example in C language where the letter `t` is either itself, or the tab symbol, when preceded with the "shift" character, the backspace, `\t`,


#### <a name=srlatch>SR Latch</a>

A bit can be stored by using hardware that can implement simple logic circuits. Consider the equations with input variable $S$ and $R$, and output variable $Q$ and $Q'$.

<div style="float:right;margin:2em;">
<a title="Goodphy, CC BY-SA 4.0 &lt;https://creativecommons.org/licenses/by-sa/4.0&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:SR_NOR_Latch_How_to_Work_Ver1_Dong-Gyu_Jang_20200309.png"><img width="334" alt="SR NOR Latch How to Work Ver1 Dong-Gyu Jang 20200309" src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/SR_NOR_Latch_How_to_Work_Ver1_Dong-Gyu_Jang_20200309.png/256px-SR_NOR_Latch_How_to_Work_Ver1_Dong-Gyu_Jang_20200309.png"></a>
</div>

\begin{eqnarray}
Q &=& \lnot\, (R \lor Q')\\
Q' &=& \lnot\, (S \lor Q)\\
\end{eqnarray}

- When $S$ and $R$ are both zero, then $Q = \lnot\,Q'$, for either $Q=0$ or $Q=1$. This is the _hold_ state.
- When $S=1$ and $R=0$, then the unique solution to the equation is $Q=1$ and $Q'=0$. This sigal is called _set_.
- When $S=0$ and $R=1$, then the unique solution to the equation is $Q=0$ and $Q'=1$. This sigal is called _reset_.
- The case where $S$ and $R$ are both 1 is not allowed in practice.

Such logic circuits were possible with early electronics, and now can be built with either transitors or Op Amps. When electronic circuits were miniturized, it became possible to store tens or hundreds of bits using SR latches.

The SR Latch works as follows. It is found in the hold state ($R=S=0$) in either the $Q=0$ or $Q=1$ state. Because the $R$ and $S$ are zero, we have a loop in which $Q$ sets $Q'$ to its negation and $Q'$ sets $Q$ to its negation. So we have a stable stored value.

To set $Q$ to 0, reset the latch. This is accomplished by setting $R=1$ then releasing $R$ back to zero. 

To set $Q$ to 1, set the latch. This is accomplished by setting $S=1$ then releasing $S$ back to zero. 



#### <a name=codon>Codon sequences</a>


#### <a name=qubit>A qubit</a>


### <a name=symbol>Ways to encode a symbol</a>

We have discussed,

- the notion of a _register_, a bank of settable bits. 
- the natural correspondence between the bit setting and an integer, but of limited range (and only positive).
- some ways to implement bits and a register.
- techniques of setting symbols into a register, for example, Baudot using shift codes.

A mechanism to protect the data in a register by making the data self-confirming. There are two versions of this self-confirmation: _error detection_ and _error correction_.

With error detection, the data element can be tested for correctness. Some bit patterns are correct and some are not. This reduces the amount of information carreid by the register, as only the correct patterns are accepted, all incorrect patterns are ignotre.

With error correction, a guess can be made about the unacceptible bit patterns to nudge them back to correct. 


#### <a name=paritycheck>Parity check</a>

We give an example parity check for a byte. We will divide the span of values of the byte into half valid and half invalid. The difference between a valid byte value and an invalid byte value will be the flip of one bit. This protects the byte against the sort of error that flips one bit. Given a valid byte, no matter which bit is flipped, the result is an invalid byte.

The parity of a register is whether the number of one's in the register is even or odd. Even parity check considers only those patterns with even parity are valid. Odd parity reverses the convention. 


In [6]:
%%file parity-check.c
#include<stdio.h>
#include<stdlib.h>

char parity_check(char c, int direction) {
    int mask = 0x01 ;
    int bit_number ;
    int parity = 0 ;
    for (bit_number=0;bit_number<8;bit_number++) {
        if (c&mask) parity++ ;
        mask <<= 1 ;
    }
    c = (parity%2==direction)? c : c^0x80 ;
    return c ;
}

int main(int argc, char * argv[]) {
    unsigned char uc = (unsigned char) atoi(argv[1]) ;
    unsigned char uc_chk ;
    uc_chk = parity_check(uc,0) ;
    printf("0x%02x: 0x%02x\n", (int) uc, (int) uc_chk ) ;
    return 0 ;
}

Writing parity-check.c


In [7]:
program = "parity-check"
!cc -o {program} {program}.c
print("code  parity")
print("------------")
for test_value in [0,1,2,3,0x10,0x30, 0x1c]:
    !./{program} {test_value}
!rm {program}.c {program}

code  parity
------------
0x00: 0x00
0x01: 0x81
0x02: 0x82
0x03: 0x03
0x10: 0x90
0x30: 0x30
0x1c: 0x9c


#### <a name=repetition>Repetition code</a>

A simple error correcting code depends on repeating the symbol and taking the majority value, if there are inconsistencies among the copies. For a simple one bit of symbol and 3 bits of code, the only two valid codes are `0x0` and `0x7`. The remining 6 codes are invalid. 

The Hamming code is an error correcting code. Given to register values, the _Hamming distance_ between the values is the number of bits on which they disagree. 

For each invalid code, it is closer to one of `0x0` or `0x7` then the other, in the Hamming distance. Error correct by replacing the invalid code with the closer of the two codes.



In [5]:
%%file repetition-code-3.c
#include<stdio.h>
#include<stdlib.h>

char repetiton_check(char c) {
    int mask = 0x01 ;
    int bit_number ;
    int parity = 0 ;
    for (bit_number=0;bit_number<8;bit_number++) {
        if (c&mask) parity++ ;
        mask <<= 1 ;
    }
    c = (parity>1)? 0x7 : 0x0 ;
    return c ;
}

int main(int argc, char * argv[]) {
    unsigned char uc = (unsigned char) atoi(argv[1]) ;
    unsigned char uc_chk ;
    uc_chk = repetiton_check(uc) ;
    printf("0x%02x: 0x%02x\n", (int) uc, (int) uc_chk ) ;
    return 0 ;
}

Writing repetition-code-3.c


In [6]:
program = "repetition-code-3"
!cc -o {program} {program}.c
print("code  parity")
print("------------")
for test_value in range(8):
    !./{program} {test_value}
!rm {program}.c {program}

code  parity
------------
0x00: 0x00
0x01: 0x00
0x02: 0x00
0x03: 0x07
0x04: 0x00
0x05: 0x07
0x06: 0x07
0x07: 0x07


#### <a name=hamming>Hamming code</a>


The repetition code works to error correct, but its _code rate_ is very slow. Every 3 bits only communicates 1 bit of information, so it is a $(3,1)$ code. Codes exist with more favorable code rates. The Hamming code is a $(7,4)$ code, for every 7 bits contains 4 bits of information. The remaining 3 bits are parity bits and support the error correction.

The magic of this is that given any two valid code points, the Hamming distance is at least 3. Hence for any invalid code point there is exactly one valid point at Hamming distance of 1 from it. Otherwise said, either a code point is valid, or there there is one and only one bit to flip in the invalid point to make it valid. It is assumed that the error was the flip of this bit. We correct and move on.

And here is an image of how this is done,

<pre>
             (2)
              |
       (3)---[P]---(6)
        |   / | \   |
        |  /  |  \  |
        | /   |   \ |
        |/    |    \|
 (1)---[P]---(7)---[P]---(4)
         \         /
          \       /
           \     /
            \   /
             (5)
</pre>
Here is how to work this diagram. There are 7 bits, numbered 1 through 7, and they are represented by the parenthesized numbers, such as `(7)`. The brackets `[P]` connect to these bits in various combinations. The bits should be set so that each of these combinations has even parity.

Write the 4 bits you want to transmit into the bits 3, 5, 6 and 7. Now set bits 1, 2 and 4 so that the any P is surrounded by an even number of 1's. This is the correct code word.

__Error correction:__ Write the 7 bits into the locations 1 through 7. If all the P nodes are surrounded by an even number of 1's, the code word is correct. Else the pattern of violations of this can be resolved by the flipping of exactly one of the bits 1 through 7. Flip this and you have corrected the error.

It is obvious that there is always a bit to flip to fix the parities, and only one such bit, because each and every of the subsets of the the parity boxes has a specific bit number that touches exactly that subset.

- Bits 1, 2 and 4 touch only one parity bit. Note that the binary represention of the bit number has exactly one bit set.
- Bits 3, 6 and 5 touch all ways to take pairs of parity bits. Note that the binary representation of the bit number has exactly two bits set.
- Bit 7 touches the full set of three parity bits. Note that 7 in binary has three bits set.

The alignment is made so that if there is a parity error, it is in bit,

$$
b = \oplus_{b_i==1} [i]_b
$$

where $[i]_b$ is the binary representation of integer $i$. If there is no error, $b=0$.





In [7]:
%%file hamming-7-4.c
#include<stdio.h>
#include<stdlib.h>
#include<assert.h>

unsigned char hamming_check(char c) {
    int mask = 0x01 ;
    int bit_number ;
    int parity = 0 ;
    for (bit_number=0;bit_number<7;bit_number++) {
        if (c&mask) parity ^= (bit_number+1) ;
        mask <<= 1 ;
    }
    return parity ;
}

unsigned char hamming_encode(char m) {
    char code = 0 ;
    int i, j ;
    int parity ;
     
    for (i=0;i<4;i++) {
        int place[] = {0x04,0x10,0x20,0x40} ;
        j = 0x01<<i ;
        if (m&j) code |= place[i] ;
    }
    parity = hamming_check(code) ;
    for (i=0;i<3;i++) {
        int place[] = {0x01, 0x02, 0x08} ;
        j = 0x01<<i ;
        if (parity&j) {
            code |= place[i] ;
        }
    }
    assert(!hamming_check(code)) ;
 
    return code ;
}

int main(int argc, char * argv[]) {
    unsigned char uc = (unsigned char) atoi(argv[1]) ;
    unsigned char uc_chk, uc_code ;
    uc_code = hamming_encode(uc) ;
    printf("0x%02x: 0x%02x\n", (int) uc, (int) uc_code ) ;
    return 0 ;
}

Writing hamming-7-4.c


In [8]:
program = "hamming-7-4"
!cc -o {program} {program}.c
print("data  code")
print("----------")
for test_value in range(16):
    !./{program} {test_value}
!rm {program}.c {program}

data  code
----------
0x00: 0x00
0x01: 0x07
0x02: 0x19
0x03: 0x1e
0x04: 0x2a
0x05: 0x2d
0x06: 0x33
0x07: 0x34
0x08: 0x4b
0x09: 0x4c
0x0a: 0x52
0x0b: 0x55
0x0c: 0x61
0x0d: 0x66
0x0e: 0x78
0x0f: 0x7f


#### <a name=correction>Extended Discussion</a>

The $(7,4)$ Hamming is one member of the Hamming code family. For every $k$ there is a $(2^k-1,2^k-k-1)$ Hamming code. The way they are built are but extension of the $(7,4)$. For $k$, write $k$ partity boxes and assign a bit to every non-empty subset of the set of parity boxes. This would be $2^k-1$ bits. Write the data into the bits that are not parity, and correct by the setting of the parity boxes. The number of data bits is $2^k-1-k$. 


__Exercise:__ Write code for the $(15,11)$ Hamming code.

If we put $k=2$, we have the $(3,1)$ hamming code. This is the repetition code we talk about above. It
has graph as follows,
<pre>
(3,1): (1)--[P]--(3)--[P]--(2)
</pre>

### END